In [42]:
import pandas as pd
import os
import crew_optimizer
from openai import OpenAI
import json

In [43]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [44]:
assignment = pd.read_json("assignments/assignment.json")
assignment = assignment['assignment'][4:254]
assignment

FR778    {'captain': 'C26', 'first_officer': 'FO12', 'd...
FR432    {'captain': 'C8', 'first_officer': 'FO2', 'dea...
FR272    {'captain': 'C43', 'first_officer': 'FO37', 'd...
FR933    {'captain': 'C40', 'first_officer': 'FO16', 'd...
FR381    {'captain': None, 'first_officer': None, 'dead...
                               ...                        
C23                                                    NaN
C24                                                    NaN
C25                                                    NaN
C26                                                    NaN
C27                                                    NaN
Name: assignment, Length: 250, dtype: object

In [45]:
# Convert assignment to string for LLM analysis
# Convert pandas Series to dict to make it JSON serializable
assignment_dict = assignment.to_dict()
assignment_text = json.dumps(assignment_dict, indent=2)
print("Assignment converted to JSON string for LLM analysis")
print(f"Text length: {len(assignment_text)} characters")
print(f"Text lines: {len(assignment_text.split(chr(10)))} lines")
print("\nFirst 500 characters:")
print("=" * 50)
print(assignment_text[:500])
print("..." if len(assignment_text) > 500 else "")

Assignment converted to JSON string for LLM analysis
Text length: 20602 characters
Text lines: 1140 lines

First 500 characters:
{
  "FR778": {
    "captain": "C26",
    "first_officer": "FO12",
    "dead_heading": []
  },
  "FR432": {
    "captain": "C8",
    "first_officer": "FO2",
    "dead_heading": []
  },
  "FR272": {
    "captain": "C43",
    "first_officer": "FO37",
    "dead_heading": []
  },
  "FR933": {
    "captain": "C40",
    "first_officer": "FO16",
    "dead_heading": []
  },
  "FR381": {
    "captain": null,
    "first_officer": null,
    "dead_heading": []
  },
  "FR569": {
    "captain": null,
    "firs
...


In [46]:
openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
print(openai_api_key[:8])
print(gemini_api_key[:8])

sk-Adudq
AIzaSyCo


In [52]:
client = OpenAI(api_key=openai_api_key)

system_message = "You are an experienced crew-scheduler training an AI heuristic search - please analyse the following potential assignment and point out the biggest flaw"

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": assignment_text}]

flaw = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

print(flaw.choices[0].message.content)

The biggest flaw in this potential assignment is that there are crew members assigned to flight segments without specifying their roles. This can lead to confusion and errors in crew scheduling. For example, in the assignments for flights such as "FR381", "FR251", "FR298", "FR330", "FR697", "FR568", "FR191", "FR597", "FR560", "FR168", "FR293", "FR760", "FR955", "FR890", "FR141", "FR881" etc., crew roles are not specified which could result in improper crew allocation and disrupt flight operations.

It is crucial to always ensure that each crew member assigned to a flight segment is designated a specific role (captain, first officer, etc.) to avoid confusion and maintain the efficiency and safety of flight operations.


In [54]:
system_message = "You are an experienced crew-scheduler training an AI heuristic search - suggest an LNS destroy or repair operator"
system_message = system_message + str(flaw)

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": assignment_text}]

response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

print(response.choices[0].message.content)

One possible Large Neighborhood Search (LNS) destroy operator you could consider for this crew scheduling problem is to randomly remove crew members from assigned flights without roles specified. This operator would simulate the scenario where some crew members unexpectedly become unavailable for certain flights, and you would need to reassign roles to other crew members or find replacements.

Here's a high-level outline of the LNS destroy operator:
1. Randomly select a flight segment where crew roles are not specified.
2. Remove the assigned crew members from that flight segment and unset their roles.
3. Repeat this process for a configurable number of iterations to create a diversified set of destroyed solutions for the LNS algorithm to explore.

By using this destroy operator in conjunction with appropriate repair operators, you can guide the search algorithm to explore new solutions by repairing the destroyed assignments and improving the overall quality of crew schedules.
